#  Example with Stanford SR830 Bundle

In [1]:
# IMPORTS

#%matplotlib notebook
import qcodes as qc
import os
import numpy as np
import time
#import progressbar
from time import sleep
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from SR830_ext import SR830_ext
from bundle_lockin import BundleLockin, do2d_multi
from qcodes.instrument.base import Instrument
from qcodes.utils.validators import Numbers, Arrays
from qcodes.utils.dataset import doNd



from qcodes import initialise_or_create_database_at, load_or_create_experiment, load_by_id, Measurement
tutorial_db_path = os.path.join(os.getcwd(), 'tutorial_doND.db')
initialise_or_create_database_at(tutorial_db_path)
load_or_create_experiment(experiment_name='tutorial_exp', sample_name="no sample")

tutorial_exp#no sample#1@C:\Users\Farzad\Documents\Rasmus\SR830ext\tutorial_doND.db
-----------------------------------------------------------------------------------
1-results-1-lockin_setpoints,lockin_trace-1
2-results-2-gen2_v_now,lockin_setpoints,lockin_trace-20
3-results-3-lockin_setpoints,lockin_trace-1
4-results-4-gen2_v_now,lockin_setpoints,lockin_trace-20
5-results-5-lockin_setpoints,lockin_trace-0
6-results-6-lockin_setpoints,lockin_trace-0
7-results-7-lockin_setpoints,lockin_trace-1
8-results-8-gen2_v_now,lockin_setpoints,lockin_trace-20
9-results-9-lockin_setpoints,lockin_trace-1
10-results-10-lockin_setpoints,lockin_trace-1
11-results-11-gen2_v_now,lockin_setpoints,lockin_trace-20
12-results-12-lockin_setpoints,lockin_trace-1
13-results-13-gen2_v_now,lockin_setpoints,lockin_trace-20
14-results-14-slow_v_now,bundle_setpoints,bundle_trace_lockin-0
15-results-15-slow_v_now,bundle_setpoints,bundle_trace_lockin-0
16-results-16-slow_v_now,bundle_setpoints,bundle_trace_lockin-0


In [2]:
class DummyGenerator(Instrument):

    def __init__(self, name, **kwargs):

        super().__init__(name, **kwargs)

        self.add_parameter('v_start',
                           initial_value=0,
                           unit='V',
                           label='v start',
                           vals=Numbers(0,1e3),
                           get_cmd=None,
                           set_cmd=None)

        self.add_parameter('v_stop',
                           initial_value=1,
                           unit='V',
                           label='v stop',
                           vals=Numbers(1,1e3),
                           get_cmd=None,
                           set_cmd=None)
        
        self.add_parameter('v_now',
                           initial_value=0,
                           unit='V',
                           label='v_now',
                           vals=Numbers(self.v_start(),self.v_stop()),
                           get_cmd=None,
                           set_cmd=None)

In [3]:
slow = DummyGenerator('slow')
fast = DummyGenerator('fast')

In [4]:
# CONNECT TO INSTRUMENT

sr = SR830_ext('lockin', 'GPIB0::1::INSTR')

Connected to: Stanford_Research_Systems SR830 (serial:s/n40974, firmware:ver1.07) in 0.21s


In [5]:
# CONNECT TO INSTRUMENT

sr2 = SR830_ext('lockin2', 'GPIB0::2::INSTR')

Connected to: Stanford_Research_Systems SR830 (serial:s/n70597, firmware:ver1.07) in 0.06s


In [6]:
sr.ch1_display('X')
sr.ch1_ratio('none')
sr.buffer_SR('Trigger')
sr.buffer_trig_mode.set('ON')
sr2.ch1_display('X')
sr2.ch1_ratio('none')
sr2.buffer_SR('Trigger')
sr2.buffer_trig_mode.set('ON')
#sr.set_sweep_parameters(fast.v_now, 0,0.5,20,0.2, label="Voltage")

In [7]:
lockins = (sr,sr2)
bundle = BundleLockin('bundle',lockins)


In [10]:
do2d_multi(param_slow = slow.v_now,
           start_slow = 0,
           stop_slow = 0.5,
           num_points_slow = 40,
           delay_slow = 0.005,
           param_fast = fast.v_now,
           start_fast = 0,
           stop_fast = 0.5,
           num_points_fast = 40, 
           delay_fast = 0.05,
           bundle = bundle,
           threading=[True,False,False,True]
              )

Starting experimental run with id: 94. 
Have finished the measurement in 106.18482740000002 seconds. run_id 94
Time used in buffer reset 0.1375211999997532. Time used in send trigger 5.443418099998553. Time used in get trace 5.258122699999888
time in the fast loop 100.06767869999976
time setting in the fast loop 94.5977684
